In [73]:
import torch

> A PyTorch computational graph is a Directed Acyclic Graph (DAG) that represents the sequence of operations performed on tensors during the forward pass of a model. This graph is dynamically built during runtime, meaning it is constructed on the fly as operations are executed.

In [74]:
x = torch.tensor(3.0, requires_grad=True) # Enables the differentation wrt this variable
x # A scalar

tensor(3., requires_grad=True)

In [75]:
y = x**2
y

tensor(9., grad_fn=<PowBackward0>)

> `backward` - Computes the gradient of current tensor wrt graph leaves. The graph is differentiated using the chain rule.

In [76]:
y.backward()
# Computational Graph: X -> square function -> y(leaf node)

In [77]:
x.grad
# Differentation of leaf node wrt x

tensor(6.)

In [78]:
x = torch.tensor(4.0, requires_grad=True)

y = 2 * x * (torch.cos(x) ** 2) # Multiplication Rule of Derivative

z = torch.sin(y)

In [79]:
x

tensor(4., requires_grad=True)

In [80]:
y

tensor(3.4180, grad_fn=<MulBackward0>)

In [81]:
z

tensor(-0.2729, grad_fn=<SinBackward0>)

> `Computational Graph`: X  →  func(2 * x * cos(x))  →  y  →  func(sin(y))  →  z

In [82]:
z.backward()

In [83]:
x.grad

tensor(6.7924)

In [84]:
y.grad

/tmp/ipython-input-84-486760323.py:1: UserWarning: The .grad attribute of a Tensor that is not a leaf Tensor is being accessed. Its .grad attribute won't be populated during autograd.backward(). If you indeed want the .grad field to be populated for a non-leaf Tensor, use .retain_grad() on the non-leaf Tensor. If you access the non-leaf Tensor by mistake, make sure you access the leaf Tensor instead. See github.com/pytorch/pytorch/pull/30531 for more informations. (Triggered internally at /pytorch/build/aten/src/ATen/core/TensorBody.h:489.)
  y.grad


> Gradients of non leaf node will not be calculated

---

In [85]:
# Inputs
x = torch.tensor(6.7)  # Input feature
y = torch.tensor(0.0)  # True label (binary)

w = torch.tensor(1.0)  # Weight
b = torch.tensor(0.0)  # Bias

In [86]:
# Binary Cross-Entropy Loss for scalar
def binary_cross_entropy_loss(prediction, target):
    epsilon = 1e-8  # To prevent log(0)
    prediction = torch.clamp(prediction, epsilon, 1 - epsilon)
    return -(target * torch.log(prediction) + (1 - target) * torch.log(1 - prediction))

In [87]:
# Forward pass
z = w * x + b  # Weighted sum (linear part)
y_pred = torch.sigmoid(z)  # Predicted probability

# Compute binary cross-entropy loss
loss = binary_cross_entropy_loss(y_pred, y)

In [88]:
loss

tensor(6.7012)

In [89]:
# Derivatives:
# 1. dL/d(y_pred): Loss with respect to the prediction (y_pred)
dloss_dy_pred = (y_pred - y)/(y_pred*(1-y_pred))

# 2. dy_pred/dz: Prediction (y_pred) with respect to z (sigmoid derivative)
dy_pred_dz = y_pred * (1 - y_pred)

# 3. dz/dw and dz/db: z with respect to w and b
dz_dw = x  # dz/dw = x
dz_db = 1  # dz/db = 1 (bias contributes directly to z)

# Chain Rule
dL_dw = dloss_dy_pred * dy_pred_dz * dz_dw
dL_db = dloss_dy_pred * dy_pred_dz * dz_db

In [90]:
print(f"Manual Gradient of loss w.r.t weight (dw): {dL_dw}")
print(f"Manual Gradient of loss w.r.t bias (db): {dL_db}")

Manual Gradient of loss w.r.t weight (dw): 6.691762447357178
Manual Gradient of loss w.r.t bias (db): 0.998770534992218


In [91]:
x = torch.tensor(6.7)
y = torch.tensor(0.0)

w = torch.tensor(1.0, requires_grad=True)
b = torch.tensor(0.0, requires_grad=True)

In [92]:
# Forward Pass
z = w*x + b
z

tensor(6.7000, grad_fn=<AddBackward0>)

In [93]:
# Predict Probabilities
y_pred = torch.sigmoid(z)
y_pred

tensor(0.9988, grad_fn=<SigmoidBackward0>)

In [94]:
loss = binary_cross_entropy_loss(y_pred, y)
loss

tensor(6.7012, grad_fn=<NegBackward0>)

> `Computational Graph`: w, b  →  foreard pass(x, w, b)  →  z  →  probabilities(z)  →  y_pred  →  loss(y_pred, y_actual)

In [95]:
loss.backward()

In [96]:
print(w.grad)
print(b.grad)

tensor(6.6918)
tensor(0.9988)


---

In [97]:
# Gradients with multiple X values
x = torch.tensor([[1.0, 2.0, 3.0], [2.5, 9.8, 5.7]], requires_grad = True)
x

tensor([[1.0000, 2.0000, 3.0000],
        [2.5000, 9.8000, 5.7000]], requires_grad=True)

In [98]:
y = (x**2).mean()
y

tensor(24.7967, grad_fn=<MeanBackward0>)

In [99]:
y.backward()

In [100]:
x.grad

tensor([[0.3333, 0.6667, 1.0000],
        [0.8333, 3.2667, 1.9000]])

---

In [101]:
# clearing grad
x = torch.tensor(2.0, requires_grad = True)
x

tensor(2., requires_grad=True)

In [102]:
y = x ** 2
y

tensor(4., grad_fn=<PowBackward0>)

In [103]:
y.backward()

In [104]:
x.grad

tensor(4.)

> The problem with executing `x.grad` muptiple times will store the gradient values in the memory → It you run this two `x.grad` two times then ouptut will be `d1 + d2` - The values of gradients will get added

In [105]:
# Inplace operation
x.grad.zero_()

tensor(0.)

---

## `Disable gradient tracking`

In [106]:
x = torch.tensor(2.0, requires_grad=True)
x

tensor(2., requires_grad=True)

In [107]:
y = x ** 2
y

tensor(4., grad_fn=<PowBackward0>)

In [108]:
y.backward()

In [109]:
x.grad

tensor(4.)

> 1. requires_grad_(False)
> 2. detach()
> 3. torch.no_grad()

In [110]:
x.requires_grad_(False) # requires_grad = False (_ inplace)

tensor(2.)

In [111]:
x

tensor(2.)

In [112]:
y = x ** 2

In [113]:
y

tensor(4.)

In [114]:
try:
    y.backward()
except Exception as e:
    print(e)

element 0 of tensors does not require grad and does not have a grad_fn


In [115]:
x = torch.tensor(2.0, requires_grad=True)
x

tensor(2., requires_grad=True)

In [116]:
y = x ** 2

In [117]:
y

tensor(4., grad_fn=<PowBackward0>)

In [118]:
y.backward()
x.grad

tensor(4.)

In [119]:
z = x.detach()
z

tensor(2.)

In [120]:
y = z ** 2
y

tensor(4.)

In [121]:
try:
    y.backward()
except Exception as e:
    print(e)

element 0 of tensors does not require grad and does not have a grad_fn
